In [2]:
%reload_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.algorithms.components import strongly_connected_component_subgraphs
from networkx.algorithms.components import connected_component_subgraphs
import timeit
import os
from bitarray import bitarray

import pickle
import time

workdir = "/home/michele/aalto/dm/DMproject/dmproject/approximate"
if(os.curdir != workdir):
    os.chdir("/home/michele/aalto/dm/DMproject/dmproject/approximate")

import sys
sys.path.insert(0, '/home/michele/aalto/dm/DMproject/dmproject/utils')

import utils
graph_names = ["wikivote", "epinions", "gplus", "pokec"]

In [5]:
graph_files = []
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'wiki_vote', graph_names[0] + '_cc.edgelist'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'epinions', graph_names[1] + '_cc.edgelist'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'gplus', graph_names[2] + '_cc.edgelist'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'soc_pokec', graph_names[3] + '_cc.edgelist'))

np.random.seed(132435)
p_samples = [5, 10, 15, 20]
n_repetitions = [3, 5, 7]


stats_cc_g = []
for graph_file, graph_name in zip(graph_files[2:3], graph_names[2:3]):
    #graph_file = graph_files[0]
    #graph_name = graph_names[0]
    k = 4
    r = 10
    h_max = 10
    
    G = nx.read_adjlist(graph_file, create_using=nx.Graph())
    print("graph " + graph_name + "loaded")
    n = len(G.nodes)
    selected_nodes = utils.reservoir_sampling_nodes(G, [1], [n], False)[0][0][0]
    
    l = int(np.log(n))+r
    bitmap = np.ndarray((len(selected_nodes), k, h_max, l), dtype=int)
    
    
    index_node_dict = dict()
    for n, node in enumerate(selected_nodes):
        index_node_dict[node] = n
    
    
    print(bitmap.shape)
    inf_map = np.zeros((len(selected_nodes), h_max))
    
    for node in index_node_dict:
        M = bitarray()
        for i in range(k):
            a = bitarray()
            for j in range(l):
                prob = 1.0/(2**(j+1))
                if np.random.random() < prob:
                    a.append(True)
                else:
                    a.append(False)
            for b in range(len(a)):
                bitmap[index_node_dict[node], i, 0, b] = a[b]
        
    for h in range(1,h_max):
        for node in index_node_dict:
            n = index_node_dict[node]
            for i in range(k):
                bitmap[n, i, h] = bitmap[n, i, h-1] 
            for neighbor in G.neighbors(node):
                if neighbor in index_node_dict:
                    m = index_node_dict[neighbor]
                    for i in range(k):
                        tmp_n = bitarray(bitmap[n, i, h].tolist())
                        tmp_m = bitarray(bitmap[m, i, h-1].tolist())
                        for b in range(l):
                            bitmap[n, i, h, b] = tmp_n[b] or tmp_m[b]
                    
        for node in index_node_dict:
            n = index_node_dict[node]
            avg_b = 0
            for i in range(k):
                try:
                    b = bitarray(bitmap[n, i, h].tolist()).index(False)
                except ValueError:
                    b = l
                avg_b += b
            avg_b = avg_b/float(k)
            individual_neighborhood_f = (2**b)/0.77351
            inf_map[n, h] = individual_neighborhood_f
        
        # check if we have to terminate the computation
        for node in index_node_dict:
            n = index_node_dict[node]
            terminate = True
            if inf_map[n,h] >= inf_map[n,h-1]-0.1 and inf_map[n,h] <= inf_map[n,h-1]+0.1:
                terminate = terminate and True
            else:
                terminate = terminate and False
        
        if terminate:
            break
        print(h)
        
    


graph gplusloaded
(69501, 4, 10, 21)


1


2


KeyboardInterrupt: 

In [55]:
#for i in range(len(selected_nodes)):
#    for j in range(h_max):
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)}, threshold=sys.maxsize)        

with open("output.txt", "w") as f:
    f.write(str(inf_map))